In [1]:
import pandas as pd
import numpy as np
import os

In [5]:
folder_path='C:/Users/511232/Desktop/DSS/DATA COLLECTOR/Received questionnaires/Population'

#check all xlsx files in the folder

xlsx_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
xlsx_files

['tunisia_population - Population_1_a.xlsx',
 'tunisia_population - Population_20_a.xlsx',
 'tunisia_population - Population_8_a.xlsx']

In [6]:
#read in all files and combine into a single dataframe
dataframes = []
for file in xlsx_files:
    df = pd.read_excel(os.path.join(folder_path, file))
    #find the first row that is all empty (NaN)
    separator_iloc = df[df.isnull().all(axis=1)].index[0]

    #for data dataframe
    df_data=df.iloc[1:separator_iloc , :].copy().reset_index(drop=True)
    #reshape the data dataframe from wide to long
    #id_vars are all colums that are not integers (years)
    id_variables = [col for col in df_data.columns if not col.isdigit()]

    df_data=df_data.melt(id_vars=id_variables, var_name='السنة', value_name='العدد')
    #convert the 'السنة' column to numeric
    df_data['السنة'] = df_data['السنة'].astype(int)
    

    #for the source dataframe
    # Find the header row for the 'source' table (df2).
    source_header=df.iloc[separator_iloc + 1, :3].tolist()
    #read in data
    df_source=df.iloc[separator_iloc + 2:separator_iloc + 18, :3].copy().reset_index(drop=True)
    #set the header
    df_source.columns=source_header
    df_source['السنة'] = df_source['السنة'].astype(int)
    
    #merge the source with the data left on المؤشر, السنة
    df_merged=pd.merge(df_data, df_source, on=['السنة'], how='left')

    # #append to data list
    dataframes.append(df_merged)
    

In [8]:
dataframes[2]

,المؤشر_x,الدولة,الجنس,المواطنة,السنة,العدد,المؤشر_y,المصدر
0,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,ذكور,مواطنون,2010,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,a
1,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,ذكور,غير مواطنين,2010,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,a
2,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,إناث,مجموع المواطنين وغير المواطنين,2010,27.6,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,a
3,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,إناث,مواطنون,2010,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,a
4,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,إناث,غير مواطنين,2010,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,a
...,...,...,...,...,...,...,...,...
115,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,إناث,مواطنون,2024,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,NaN
116,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,إناث,غير مواطنين,2024,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,NaN
117,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,كلا الجنسين,مجموع المواطنين وغير المواطنين,2024,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,NaN
118,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,تونس,كلا الجنسين,مواطنون,2024,NaN,متوسط العمر عند الزواج الأول (سنوات) حسب المواطنة,NaN
